In [ ]:
import pandas as pd
import copy
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.ticker import NullFormatter

In [ ]:
df_original = pd.read_csv("main_with_tpo.csv").drop(columns=["Tours"])
df_original.head(20)

In [ ]:
# Replace the time of the rows whose costs are infinite
### Inplace
df = copy.deepcopy(df_original)
df.loc[df["Cost"]==-1, "Time[s]"] = 0
df.loc[df["Cost"]==float('inf'), "Time[s]"] = 600

### Edit the copied df.
# df[df["Cost"] == float("inf")]["Time[s]"] = 180

In [ ]:
# df.groupby(["Node", "ConstraintRatio", "MaxTimeGap", "Agent", "Solver"])[
#     ["Time[s]", "Cost"]
# ].median().reset_index()

In [ ]:
df.loc[(df["MaxTimeGap"] == 10) & (df["ConstraintRatio"] == 0.75)]\
    .set_index(["Node", "Agent", "Solver"]).head()

In [ ]:
df.loc[(df["MaxTimeGap"] == 10) & (df["ConstraintRatio"] == 0.75)]\
    .groupby(["Node", "Agent", "Solver"])['Time[s]'].mean().head()

In [ ]:
df.loc[(df["MaxTimeGap"] == 10) & (df["ConstraintRatio"] == 0.75)]\
    .groupby(["Node", "Agent", "Solver"])['Time[s]'].mean().reset_index()\
    .set_index(["Node", "Agent", "Solver"]).unstack().head()

In [ ]:
plt.rcParams.update({"font.size": 12})


Solvers = df["Solver"].unique()
print(Solvers)

Nodes = df["Node"].unique()
print(Nodes)

ConstraintRatio = df["ConstraintRatio"].unique()
print(ConstraintRatio)
# ConstraintRatio = [0.25]

MaxTimeGap = df["MaxTimeGap"].unique()
print(MaxTimeGap)
# MaxTimeGap = [30]

Agents = df["Agent"].unique()
print(Agents)

xLabel = "MaxTimeGap"
yLabel = "ConstraintRatio"
zLabel = "Solver"
xElems = MaxTimeGap
yElems = ConstraintRatio
zElems = Solvers
evalColumn = "Time[s]"


fig, ax = plt.subplots(figsize=(12, 6))
df.groupby(["Node", "Agent", "Solver"])['Time[s]'].mean().reset_index()\
    .set_index(["Node", "Solver", "Agent"]).unstack()\
    .plot(
        kind="bar",
        y="Time[s]",
        grid=True,
        # rot=0,
        ax=ax,
        ylim=(0, 600),
        legend=False,
        logy=True,
        width=0.75,
    )
ax.set_yscale("log")

ax.grid("on", which="minor", axis="x", alpha=0.2)
ax.grid("on", which="minor", axis="y", alpha=0.2)
ax.set_ylabel("Time[s]")

consts = {c: df[c].iloc[0] for c in df.columns if len(df[c].unique()) == 1}
const_strs = [f"{k}={v}" for k, v in consts.items()]
const_str = " ".join(const_strs)

plt.subplots_adjust(wspace=0.05, hspace=0.05)
# fig.tight_layout()
plt.legend(
    bbox_to_anchor=(0.3, 0.9),
    loc="lower left",
    bbox_transform=fig.transFigure,
    ncol=len(Agents)*len(Solvers),
)
plt.savefig(f"benchmark.png", dpi=300)

In [ ]:
plt.rcParams.update({"font.size": 12})


Solvers = df["Solver"].unique()
print(Solvers)

Nodes = df["Node"].unique()
print(Nodes)

ConstraintRatio = df["ConstraintRatio"].unique()
print(ConstraintRatio)
# ConstraintRatio = [1, 0.25]

MaxTimeGap = df["MaxTimeGap"].unique()
print(MaxTimeGap)
# MaxTimeGap = [10, 50]

Agents = df["Agent"].unique()
print(Agents)

xLabel = "MaxTimeGap"
yLabel = "ConstraintRatio"
zLabel = "Solver"
xElems = MaxTimeGap
yElems = ConstraintRatio
zElems = Solvers
evalColumn = "Time[s]"


fig, axes = plt.subplots(nrows=len(xElems), ncols=len(yElems), figsize=(24, 10))
# fig, axes = plt.subplots(nrows=len(xElems), ncols=len(yElems), figsize=(16, 10))
for irow, xelem in enumerate(xElems):
    for icol, yelem in enumerate(yElems):
        # sns.boxplot(
        #     data=df.loc[(df[xLabel] == xelem) & (df[yLabel] == yelem)],
        #     x="Node",
        #     y="Time[s]",
        #     hue=df[['Agent', 'Solver',]].apply(tuple, axis=1),
        #     ax=axes[irow, icol],
        #     legend=False,
        #     palette="Set1",
        #     )
        df.loc[(df[xLabel] == xelem) & (df[yLabel] == yelem)]\
            .groupby(["Node", "Agent", "Solver"])['Time[s]'].mean().reset_index()\
            .set_index(["Node", "Solver", "Agent"]).unstack()\
            .plot(
                kind="bar",
                y="Time[s]",
                grid=True,
                # rot=0,
                ax=axes[irow, icol],
                ylim=(0, 600),
                legend=False,
                logy=True,
                width=0.75,
            )

        # sns.boxplot(
        #     data=df.loc[(df[xLabel] == xelem) & (df[yLabel] == yelem)],
        #     x="Node",
        #     y="Time[s]",
        #     hue='Agent',
        #     ax=axes[irow, icol],
        #     legend=True if irow==len(xElems)-1 and icol==len(yElems)-1 else False,
        #     palette="Set1",
        #     )
        # df.loc[(df[xLabel] == xelem) & (df[yLabel] == yelem)]\
        # .boxplot(
        #     column=[evalColumn],
        #     by=["Agent"],
        #     grid=True,
        #     rot=90,
        #     ax=axes[irow, icol],
        # )
        axes[irow, icol].set_yscale("log")

        axes[irow, icol].grid("on", which="minor", axis="x", alpha=0.2)
        axes[irow, icol].grid("on", which="minor", axis="y", alpha=0.2)

        if icol == 0:
            axis = axes[irow, icol].get_yaxis()
            axis.set_label_text(f"{xLabel}={xelem}")
        else:
            axis = axes[irow, icol].get_yaxis()
            # axis.set_ticks_position('none')
            # axis.set_label_text()
            axis.set_major_formatter(NullFormatter())
            # axis.set_visible(False)
            axes[irow, icol].set(xlabel=None)

        if irow == len(xElems) - 1:
            axis = axes[irow, icol].get_xaxis()
            axis.set_label_text(f"{yLabel}={yelem}")
        else:
            axis = axes[irow, icol].get_xaxis()
            # axis.set_label_text()
            # axis.set_ticks_position('none')
            axis.set_major_formatter(NullFormatter())
            # axis.set_visible(False)
            axes[irow, icol].set(xlabel=None)


consts = {c: df[c].iloc[0] for c in df.columns if len(df[c].unique()) == 1}
const_strs = [f"{k}={v}" for k, v in consts.items()]
const_str = " ".join(const_strs)
# fig.suptitle(f"{evalColumn} with Constants of {const_str}")
fig.suptitle(f"{evalColumn}")

plt.subplots_adjust(wspace=0.05, hspace=0.05)
# fig.tight_layout()
plt.legend(
    bbox_to_anchor=(0.4, 0.9),
    loc="lower left",
    bbox_transform=fig.transFigure,
    ncol=len(Agents)*len(Solvers),
)
plt.savefig(f"benchmark.png", dpi=300)

In [ ]:
df = pd.read_csv("main_with_tpo.csv")\
                  .drop(columns=["Tours"])
df.drop(df[df["Bound"]==-1].index)
df.head()

In [ ]:
from pandas import DataFrame
import pandas as pd
plt.rcParams.update({"font.size": 20})


df: DataFrame = pd.read_csv("main_with_tpo.csv")\
                  .drop(columns=["Tours"])
df.drop(df[df["Bound"]==-1].index)

df["Bound/Makespan"] = df["Bound"] / df["Cost"]

fig = plt.figure(figsize=(18, 6))
ax = fig.add_subplot(111)
df.boxplot(
    column="Bound/Makespan",
    # by=["Node", "Agent"],
    by="Node",
    grid=True,
    rot=90,
    ax=ax,
)
fig.suptitle("")
ax.set_title(r'Normalized Uniform Bound Timed $\epsilon$ (divided by the makespan)')
# ax.set_ylim([0, 0.5])
ax.set_yscale("log")

In [ ]:
from pandas import DataFrame


df: DataFrame = pd.read_csv("main_with_tpo.csv")\
                  .drop(columns=["Tours"])
df = df.drop(df[df["Bound"]==-1].index)
m = df['Bound'].mean()
s = df['Bound'].std()

mmi = df['Bound'].min()
mma = df['Bound'].max()

# print(f"{m:.2f}+-{s:.2f}, \in [{mmi}, {mma}]")

fig = plt.figure(figsize=(18, 6))
ax = fig.add_subplot(111)
df.hist(column="Bound", bins=50, ax=ax)
axis = ax.get_xaxis()


# OR-Tools Solver Comparison

In [ ]:
df_original = pd.read_csv("benchmark_ortoolsolvers.csv")
df_original.head()

In [ ]:
# Replace the time of the rows whose costs are infinite
### Inplace
df = copy.deepcopy(df_original)
df.loc[df["Cost"] == float("inf"), "Time[s]"] = 180
### Edit the copied df.
# df[df["Cost"] == float("inf")]["Time[s]"] = 180

In [ ]:
plt.rcParams.update({"font.size": 12})


# Get a list of arguments

Solvers = df["Solver"].unique()
print(Solvers)

Nodes = df["Node"].unique()
print(Nodes)

ConstraintRatio = df["ConstraintRatio"].unique()
print(ConstraintRatio)

MaxTimeGap = df["MaxTimeGap"].unique()
print(MaxTimeGap)

Agents = df["Agent"].unique()
print(Agents)

xLabel = "Node"
yLabel = "Agent"
xElems = Nodes
yElems = Agents

fig, axes = plt.subplots(nrows=len(xElems), ncols=len(yElems), figsize=(12, 5))

for irow, xelem in enumerate(xElems):
    for icol, yelem in enumerate(yElems):
        df.loc[(df[xLabel] == xelem) & (df[yLabel] == yelem)].boxplot(
            column=["Time[s]"],
            by=["Solver"],
            grid=True,
            rot=90,
            ax=axes[irow, icol],
        )
        # .plot(
        #     kind="box",
        #     # x="Solver",
        #     y="Time[s]",
        #     grid=True,
        #     # rot=0,
        #     ax=axes[irow, icol],
        #     ylim=(0, 180),
        #     legend=False,
        #     logy=True,
        #     # width=0.75,
        # )
        axes[irow, icol].set_yscale("log")

        axes[irow, icol].grid("on", which="minor", axis="x", alpha=0.2)
        axes[irow, icol].grid("on", which="minor", axis="y", alpha=0.2)

        if icol == 0:
            axis = axes[irow, icol].get_yaxis()
            axis.set_label_text(f"{xLabel}={xelem}")
        else:
            axis = axes[irow, icol].get_yaxis()
            axis.set_major_formatter(NullFormatter())
            axes[irow, icol].set(xlabel=None)

        if irow == len(xElems) - 1:
            axis = axes[irow, icol].get_xaxis()
            axis.set_label_text(f"{yLabel}={yelem}")
        else:
            axis = axes[irow, icol].get_xaxis()
            axis.set_major_formatter(NullFormatter())
            axes[irow, icol].set(xlabel=None)

plt.subplots_adjust(wspace=0.05, hspace=0.05)
# fig.tight_layout()
# plt.legend(
#     bbox_to_anchor=(0.3, 0.9),
#     loc="lower left",
#     bbox_transform=fig.transFigure,
#     # ncol=len(Agents),
# )
plt.savefig("benchmark_ortoolsovlers.png", dpi=300)

In [ ]:
import numpy as np

dictionary = {'Banner':['Type1']*10+['Type2']*10,
              'Northen_californina':np.random.rand(20),
              'Texas':np.random.rand(20)}
df = pd.DataFrame(dictionary)
df.head(20)

In [ ]:
df = pd.melt(df,id_vars=['Banner'],value_vars=['Northen_californina','Texas'],
             var_name='zone', value_name='amount')
df.head(20)

In [ ]:
import matplotlib.pyplot as plt
plt.figure(figsize=(9,9)) #for a bigger image
sns.boxplot(x="Banner", y="amount", hue="zone", data=df, palette="Set1")

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# Create a figure and a set of subplots
fig, axes = plt.subplots(2, 2, figsize=(10, 6))

# Generate some data
data = sns.load_dataset("iris")

# Plot the data on each subplot
sns.boxplot(
    x = "species",
    y = "sepal_length",
    showmeans=True,
    ax=axes[0, 0],
    data=data
)

sns.boxplot(
    x = "species",
    y = "sepal_width",
    showmeans=True,
    ax=axes[0, 1],
    data=data
)

sns.boxplot(
    x = "species",
    y = "petal_length",
    showmeans=True,
    ax=axes[1, 0],
    data=data
)

sns.boxplot(
    x = "species",
    y = "petal_width",
    showmeans=True,
    ax=axes[1, 1],
    data=data
)

# Add a legend to the figure
plt.legend(title="Species", loc="upper center", bbox_to_anchor=(0.5, 1.15))

# Show the plot
plt.show()


In [ ]:
import pandas as pd

df = pd.read_csv("main_with_tpo_ICRA3.csv").drop(columns=["Tours"])
df.head(20)

In [ ]:
print(df[df.Node == 160].drop([238])['Time[s]'].mean())
print(df[df.Node == 160].drop([238])['Time[s]'].std())

In [ ]:
print(df[df.Node == 160]['Time[s]'].mean())
print(df[df.Node == 160]['Time[s]'].std())